Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from matplotlib.dates import date2num
from matplotlib import dates as mdates
from matplotlib.dates import MonthLocator, DateFormatter
import plotly.express as px
from plotly.subplots import make_subplots

import data

In [ ]:
ru_personnel_loss = pd.read_csv('/Users/andriussvilpauskas/Desktop/Turing College/Python project data/russia_losses_personnel.csv')
ru_equipment_loss = pd.read_csv('/Users/andriussvilpauskas/Desktop/Turing College/Python project data/russia_losses_equipment.csv')
ru_losses_type = pd.read_csv('/Users/andriussvilpauskas/Desktop/Turing College/Python project data/losses_russia.csv')

summary statistics

In [ ]:
print(ru_personnel_loss.describe())
print(ru_equipment_loss.describe())
print(ru_losses_type.describe())

calculate daily changes for personnel and equipment losses

In [ ]:
ru_personnel_loss['daily_change'] = ru_personnel_loss['personnel'].diff()

ru_equipment_loss['date'] = pd.to_datetime(ru_equipment_loss['date'])
ru_equipment_loss.set_index('date', inplace=True)
unit_type_columns = [col for col in ru_equipment_loss.columns if col not in ['day', 'greatest losses direction', 'submarines']]
ru_equipment_loss[unit_type_columns] = ru_equipment_loss[unit_type_columns].astype(float)
ru_equipment_loss['daily_total'] = ru_equipment_loss[unit_type_columns].sum(axis=1)
ru_equipment_loss['daily_change'] = ru_equipment_loss['daily_total'].diff()

Plotting equipment losses using Plotly pie chart

In [ ]:
fig = go.Figure(data=[go.Pie(labels=ru_losses_type['equipment'], values=ru_losses_type['losses_total'], textinfo='label',
                             insidetextorientation='radial'
)])
fig.show()

Function to create subplots with common x-axis for personnel and equipment losses over time

In [ ]:
def plot_losses_over_time(ax, x, y, color, ylabel, title, marker_dates):
    ax.plot_date(pd.to_datetime(x), y, marker='', linestyle='-', color=color)
    for date, line_color in marker_dates.items():
        ax.axvline(x=date2num(pd.to_datetime(date)), color=line_color, linestyle='--', linewidth=2,label=legend_labels.pop(0) if legend_labels else None)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()

Function to create subplots for daily changes in personnel and equipment losses

In [ ]:
def plot_daily_changes(ax, x, y, color, ylabel, title, marker_dates):
    ax.plot_date(pd.to_datetime(x), y, marker='', linestyle='-', color=color)
    for date, line_color in marker_dates.items():
        ax.axvline(x=date2num(pd.to_datetime(date)), color=line_color, linestyle='--', linewidth=2,label=legend_labels.pop(0) if legend_labels else None)
    ax.set_xlabel('Date')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()

Plotting personnel and equipment losses over time

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)
marker_dates = {'2022-04-07': 'blue', '2022-08-29': 'green', '2022-11-12': 'yellow', '2023-06-08': 'purple'}
legend_labels = ['End of initial invasion', '2022 counteroffensive', 'second stalemate', '2023 counteroffensives']

plot_losses_over_time(ax1, ru_personnel_loss['date'], ru_personnel_loss['personnel'], 'r',
                       'Number of Soldiers Lost', 'Soldiers Lost Over Time', marker_dates)

plot_losses_over_time(ax2, ru_equipment_loss.index, ru_equipment_loss['daily_total'], 'black',
                       'Number of Equipment Lost', 'Equipment Lost Over Time', marker_dates)

plt.tight_layout()
plt.show()


Plotting daily changes in personnel and equipment losses

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

plot_daily_changes(ax1, ru_personnel_loss['date'], ru_personnel_loss['daily_change'], 'r',
                   'Number of Soldiers Lost', 'Soldiers Lost Over Time', marker_dates)

plot_daily_changes(ax2, ru_equipment_loss.index, ru_equipment_loss['daily_change'], 'black',
                   'Number of Equipment Lost', 'Equipment Lost Over Time', marker_dates)

plt.tight_layout()
plt.show()